In [1]:
import pymskt as mskt
from pymskt.mesh import BoneMesh, CartilageMesh
import os

In [2]:
location_seg = '../../data/right_knee_example.nrrd'

To create a mesh, we must first pass a segmentation image (simpleitk) to the mesh object. This can be done in multiple ways. 

In [3]:
# initiate the bone mesh object
femur = BoneMesh(path_seg_image=location_seg,  # path to the segmentation iamge being used
                 label_idx=5,                  # what is the label of this bone
                 list_cartilage_labels=[1])    # a list of labels for cartialge associated w/ this bone

# Create the bone mesh
femur.create_mesh(
    smooth_image_var=1.0       # This is the variance of the gaussian filter applied to binary image b4 meshing
)

# Resample the bone surface to have a specified number of nodes. 
femur.resample_surface(
    clusters=20000             # This is the number of nodes/vertices on the surface. It might vary slightly
)

# Calcualte cartialge thickness for the cartialge meshes associated with the bone
femur.calc_cartilage_thickness(
    image_smooth_var_cart=0.5   # variance for the gaussian filter applied to the binary cart meshes b4 smoothing. 
)

# assign the cartilage regions to the surface
femur.assign_cartilage_regions()

# IF you want to save the mesh you can uncomment the below line
# femur.save_mesh('path/to_save/mesh.vtk') # I choose VTK becuase it preserves the scalars - STL does not. 




/Users/gattia/opt/miniconda3/envs/imaging/lib/python3.8/site-packages/pyvista/core/dataset.py:1401: PyvistaDeprecationWarning: Use of `point_arrays` is deprecated. Use `point_data` instead.
  warnings.warn(
/Users/gattia/opt/miniconda3/envs/imaging/lib/python3.8/site-packages/pyvista/core/dataset.py:1541: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  warnings.warn(


- Below is optional, and only necessary if you want to view the meshes in your Jupyter Notebook. 
- Instructions for installing ITK Widgets is on the readme page. 

In [4]:
from itkwidgets import view

# Note that we have to get the mesh from the femur object `femur.mesh` to input it into itkwidgets. 
# In this notebook, you should be able to swap between scalars (thickness & labels)
# thickness is the cartilage thickness in mm, labels is the cartilage label of the associated thickness value. 
view(geometries=[femur.mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

- If the above surface scalars seem too "rough" for you there are a few options. 
    1. We could just smooth the bone & cartilage meshes more 
    2. We can actually smooth these scalar value - we'll show that below

In [5]:
femur.smooth_surface_scalars(
    smooth_only_cartilage=True,           # by default we dont count regions without cartilage in smoothing - we dont want them dragging down mean. 
    scalar_sigma=2.,                      # this is the standard deviation of a gaussian filter
    scalar_array_name='thickness (mm)'    # we have to specify what scalars to smooth - we might have T2 or other parameters also that use different sigma
)

- We can re-run the viewer to see how the smoothed scalars look
- We should be able to see that a lot of the noise in the thickness values has been washed out

In [6]:
view(geometries=[femur.mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [7]:
femur.save_mesh(os.path.expanduser('~/Downloads/femur_mesh.vtk'))